In [1]:
from bs4 import BeautifulSoup
from requests import TooManyRedirects
import re
import requests
from datetime import datetime
import os
import pandas as pd
import regex as re

In [2]:
re.sub(r"[^a-zA-Z]{1,}", "_", 'Hocus Pocus').lower()

'hocus_pocus'

In [3]:
re.sub(r"[^a-zA-Z]{1,}", "_", 'Venom: The Last Dance').lower()

'venom_the_last_dance'

In [4]:
def get_critic_page_init(movie_name):
    movie_name = re.sub(r"[^a-zA-Z]{1,}", "_", movie_name).lower()
    response = requests.get(f'https://www.rottentomatoes.com/m/{movie_name}/reviews')
    soup = BeautifulSoup(response.content, 'html.parser')
    critics = soup.find_all('div',class_ = 'review-row')

    # find movie id
    rtid = soup.find('script',{"id":"mps-page-integration"}).contents[0].replace('|','').strip()
    rtid = [x for x in rtid.split(',') if 'rtid' in x][0].strip().split(':')[-1][1:-1]
    return critics,rtid

In [5]:
def get_critic_page_follow(movie_id,start_token = None):
    url = f'https://www.rottentomatoes.com/napi/movie/{movie_id}/reviews/all?after={start_token}%3D%3D&pageCount=20'
    print(f'Using url:{url}')
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('Aborted as response code is not 200')
            return None
    except error as e:
        print(f'[request {url} failed] : {e}')
    return json.loads(response.content)

In [6]:
def get_review_data_soup(critics):
    critics_reviewer = critics.find_all('div',class_ = 'reviewer-name-and-publication')
    disp_name = critics_reviewer[0].find( class_ = "display-name").contents[0].strip()
    # find review_date
    critics_score_content = [x for x in critics.find('p',class_ = 'original-score-and-url').contents if hasattr(x,'contents')]
    def try_dt_map(x):
        try:
            dt = datetime.strptime(x.contents[0], '%b %d, %Y')
            return dt
        except:
            return None
    review_date = [dt for dt in list(map(try_dt_map,critics_score_content)) if bool(dt)]
    if review_date:
        review_date = review_date[0]
    else:
        review_date = 'Not found'
        
    # find score
    critics_score_str = [x for x in critics.find('p',class_ = 'original-score-and-url').contents if not hasattr(x,'contents')]
    def try_score_map(x):
        try:
            score = x.replace('|','').strip()
            return score
        except:
            return None
    review_score = [score for score in list(map(try_score_map,critics_score_str)) if bool(score)]
    if review_score:
        review_score = review_score[0].split()[-1].strip()
    else:
        review_score = 'Not found'

    # find sentiment
    review_sentiment = critics.find('score-icon-critics').attrs['sentiment']
    # find review text
    review_text = critics.find('p', class_ = 'review-text').contents[0]
    if review_text:
        review_text = review_text
    else:
        review_text = 'Not Found'
        
    return [disp_name,review_date,review_score,review_sentiment,review_text,1]
                      

In [7]:
def get_review_data_json(json_rvw_data, page = None):
    def get_json_info(jsn,key):
        if key in jsn.keys():
            return jsn[key]
        else:
            return None
        
    disp_name = get_json_info(json_rvw_data,'criticName')
    review_date = get_json_info(json_rvw_data, 'creationDate')
    review_date = datetime.strptime(json_rvw_data['creationDate'], '%b %d, %Y')
    review_score = get_json_info(json_rvw_data, 'originalScore')
    review_sentiment = get_json_info(json_rvw_data, 'scoreSentiment')
    review_text = get_json_info(json_rvw_data, 'quote')
    
    # review_date = datetime.strptime(json_rvw_data['creationDate'], '%b %d, %Y')
    # review_score = json_rvw_data['originalScore']
    # review_sentiment = json_rvw_data['scoreSentiment']
    # review_text = json_rvw_data['quote']
    return [disp_name,review_date,review_score,review_sentiment,review_text,page]

In [8]:
def get_movie_reviews(movie_title):
    # create dataframe
    df = pd.DataFrame({'movie_title':[],
                       'movie_id':[],
                       'reviewer_name':[],
                       'review_date':[],
                       'review_score':[],
                       'review_sentiment':[],
                       'review_text':[],
                       'on_page':[]
                        })
    def append_review_df(df, movie_title, movie_id, reviewer_name, review_date, review_score, review_sentiment, review_text, on_page):
        df_to_append = pd.DataFrame({'movie_title':[movie_title],
                                     'movie_id':[movie_id],
                                     'reviewer_name':[reviewer_name],
                                     'review_date':[review_date],
                                     'review_score':[review_score],
                                     'review_sentiment':[review_sentiment],
                                     'review_text':[review_text],
                                     'on_page':[on_page]
                                    })
        return df._append(df_to_append, ignore_index = True)
    # page1 init
    init_pg_critics, rtid = get_critic_page_init(movie_title)
    # ingest page1 data
    print(f'Scraping page{1} of {movie_title}')
    for row in range(len(init_pg_critics)):
        review_row = init_pg_critics[row]
        rtid = get_critic_page_init(movie_title)[1]
        review_row_data = get_review_data_soup(review_row)
        df = append_review_df(df, movie_title, rtid, *review_row_data)
    # other page
    is_first_loop = True
    while True:
        if is_first_loop:
            start_token = 'MQ'
        res = get_critic_page_follow(movie_id = rtid,start_token = start_token)
        if not res: # the movie has no more review page
            break
        pg = 2 # page count
        print(f'Scraping page{pg} of {movie_title} using start key = {start_token}')
        for row in res['reviews']:
            # print(row)
            # print('-'*10)
            review_row_data = get_review_data_json(row, page = pg)
            df = append_review_df(df, movie_title, rtid, *review_row_data)
        is_first_loop = False # Disable init var
        # check if has next page
        print(res['pageInfo']['hasNextPage'])
        if not res['pageInfo']['hasNextPage']:
            return df
            
        start_token = res['pageInfo']['endCursor'].replace('==','')
        print(f'Found next page token {start_token}')    
    
        
        
    return df

In [9]:
x = get_movie_reviews('Venom: The Last Dance')

x

Scraping page1 of Venom: The Last Dance


C:\Users\Tan\AppData\Local\Temp\ipykernel_7736\2213794791.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return df._append(df_to_append, ignore_index = True)


Using url:https://www.rottentomatoes.com/napi/movie/0f101f8c-ec09-39c4-9be0-2f9cc464d332/reviews/all?after=MQ%3D%3D&pageCount=20
Scraping page2 of Venom: The Last Dance using start key = MQ
True
Found next page token Mg
Using url:https://www.rottentomatoes.com/napi/movie/0f101f8c-ec09-39c4-9be0-2f9cc464d332/reviews/all?after=Mg%3D%3D&pageCount=20
Scraping page2 of Venom: The Last Dance using start key = Mg
True
Found next page token Mw
Using url:https://www.rottentomatoes.com/napi/movie/0f101f8c-ec09-39c4-9be0-2f9cc464d332/reviews/all?after=Mw%3D%3D&pageCount=20
Scraping page2 of Venom: The Last Dance using start key = Mw
True
Found next page token NA
Using url:https://www.rottentomatoes.com/napi/movie/0f101f8c-ec09-39c4-9be0-2f9cc464d332/reviews/all?after=NA%3D%3D&pageCount=20
Scraping page2 of Venom: The Last Dance using start key = NA
True
Found next page token NQ
Using url:https://www.rottentomatoes.com/napi/movie/0f101f8c-ec09-39c4-9be0-2f9cc464d332/reviews/all?after=NQ%3D%3D&page

,movie_title,movie_id,reviewer_name,review_date,review_score,review_sentiment,review_text,on_page
0,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Marcelo Stiletano,2024-10-30,2/5,NEGATIVE,The story advances without much conviction thr...,1.0
1,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Jake Wilson,2024-10-30,2/5,NEGATIVE,Billed as the final instalment of the Venom tr...,1.0
2,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Randy Meeks,2024-10-30,2.5/5,POSITIVE,While the antihero deserved much more than thi...,1.0
3,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Leigh Paatsch,2024-10-30,2/5,NEGATIVE,The third and final episode in the Venom saga ...,1.0
4,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Tom Beasley,2024-10-30,Not found,POSITIVE,If you can’t enjoy Tom Hardy sitting in stony ...,1.0
...,...,...,...,...,...,...,...,...
155,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Casey Chong,2024-10-23,1.5/5,NEGATIVE,A futile third Venom movie that fails to give ...,2.0
156,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Jake Cole,2024-10-23,3/4,POSITIVE,As the film progresses&#44; it consistently es...,2.0
157,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Elliott Collins,2024-10-23,2/5,NEGATIVE,In a whirlwind of chaotic creativity&#44; Veno...,2.0
158,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Soren Andersen,2024-10-23,1.5/4,NEGATIVE,“The Last Dance” brings nothing new to the ser...,2.0


In [10]:
# for i in range(len(get_critic_page_init('hocus_pocus')[0])):
#     c = get_critic_page_init('hocus_pocus')[0][i]
#     rtid = get_critic_page_init('hocus_pocus')[1]
#     print(get_review_data_soup(c))

In [11]:
# for i in get_critic_page_follow(movie_id = rtid,start_token = 'MQ')['reviews']:
#     print(get_review_data_json(get_critic_page_follow(movie_id = rtid,start_token = 'MQ')['reviews'][i]))

In [12]:
# rr = get_critic_page_follow(movie_id = rtid,start_token = 'Mg')
# #soup = BeautifulSoup(get_critic_page_follow(movie_id = rtid,start_token = 'MQ'), 'html.parser')

In [13]:
# rr['reviews'][0]

In [14]:
# if bool(rr['pageInfo']['hasNextPage']):
#     print(666)
# else:
#     print(888)

In [15]:
# rr['reviews'][0]

In [16]:
# get_review_data_json(rr['reviews'][0])

In [17]:
# datetime.strptime(rr['reviews'][0]['creationDate'], '%b %d, %Y')